In [1]:
import pandas as pd
from pandas import DataFrame
import os
import sys
import csv
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
plt.show()
import glob
import seaborn as sns

os.chdir('/mnt/scgc/simon/microg2p/analyses/20210325_GoM_recluster/20210325_GoM_recluster_analysis/chl_analysis/')

outdir='/mnt/scgc/simon/microg2p/analyses/20210325_GoM_recluster/20210325_GoM_recluster_analysis/chl_analysis/'
if os.path.isdir(outdir):
    print(outdir, "already exists.")
else:
    os.mkdir(outdir)
    print(outdir, "is created.")

images='/mnt/scgc/simon/microg2p/analyses/20210325_GoM_recluster/20210325_GoM_recluster_images/chl_analysis'
if os.path.isdir(images):
    print(images, "already exists.")
else:
    os.mkdir(images)
    print(images, "is created.")
    
matplotlib.__version__

/mnt/scgc/simon/microg2p/analyses/20210325_GoM_recluster/20210325_GoM_recluster_analysis/chl_analysis/ already exists.
/mnt/scgc/simon/microg2p/analyses/20210325_GoM_recluster/20210325_GoM_recluster_images/chl_analysis already exists.


'3.3.3'

rerun on 2021-12-07 after correcting for salinity of media samples

In [2]:


#SAG abundance. I need to convert this to cells/ml
SAG_abundance=pd.read_csv('/mnt/scgc/simon/microg2p/analyses/20210325_GoM_recluster/Summary_files/Syto9_SAG_abundances.csv',
                         index_col=0)

#key to only select bacteriodetes genera
classification=pd.read_csv('/mnt/scgc/simon/microg2p/analyses/20210325_GoM_recluster/Summary_files/Genus_statistics.csv',
                          index_col=0)
classification=classification.rename(columns={'GTDB_genus':'GTDB_classification'})
classification=classification[['Phyla', 'Family', 'GTDB_classification']]

# chlorophyl concentration
chl_concentration=pd.read_csv('G2P_DS_physical_chl_extract_20181030_20200616.csv')


classification


,Phyla,Family,GTDB_classification
0,NaN,NaN,Unclassified
1,Proteobacteria,Pelagibacteraceae,Pelagibacter
2,Verrucomicrobiota,Akkermansiaceae,SW10
3,Proteobacteria,D2472,SCGC-AAA076-P13
4,Proteobacteria,D2472,D2472
...,...,...,...
212,Verrucomicrobiota,Opitutaceae,UBA6669
213,Planctomycetota,Pirellulaceae,UBA721
214,Proteobacteria,UBA4486,UBA7359
215,Bacteroidota,Flavobacteriaceae,UBA7446


In [3]:
# the total number of cells/ml from Syto9 stains.
cells_1704=1545111
cells_1708=2690141
cells_1711=1550260
cells_1810=1452500
cells_1904=1642105
cells_1907=2915238

In [4]:
SAG_abundance=SAG_abundance.merge(classification, on='GTDB_classification', how='left')
Bacteriodetes=SAG_abundance[SAG_abundance['Phyla']=='Bacteroidota']

Bacteriodetes['1704_cells']=Bacteriodetes['170412_SAG_abundnance']*cells_1704
Bacteriodetes['1708_cells']=Bacteriodetes['170818_SAG_abundnance']*cells_1708
Bacteriodetes['1711_cells']=Bacteriodetes['171102_SAG_abundnance']*cells_1711
Bacteriodetes['1810_cells']=Bacteriodetes['181030_SAG_abundnance']*cells_1810
Bacteriodetes['1904_cells']=Bacteriodetes['190402_SAG_abundnance']*cells_1904
Bacteriodetes['1907_cells']=Bacteriodetes['190709_SAG_abundnance']*cells_1907

#sum up all the bacterodiota cells and put into a series
bacter_1704=Bacteriodetes['1704_cells'].sum()
bacter_1708=Bacteriodetes['1708_cells'].sum()
bacter_1711=Bacteriodetes['1711_cells'].sum()
bacter_1810=Bacteriodetes['1810_cells'].sum()
bacter_1904=Bacteriodetes['1904_cells'].sum()
bacter_1907=Bacteriodetes['1907_cells'].sum()
counts=[bacter_1704, bacter_1708, bacter_1711, bacter_1810, bacter_1904, bacter_1907]
counts

/mnt/scgc/scgc_nfs/opt/common/anaconda3a/envs/jacob_env/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/mnt/scgc/scgc_nfs/opt/common/anaconda3a/envs/jacob_env/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/mnt/scgc/scgc_nfs/opt/common/anaconda3a/envs/jacob_env/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be se

[354615.63934426225,
 501891.97761194035,
 220657.44525547448,
 56258.802816901414,
 259597.48161764705,
 361891.6137931035]

In [5]:
bacteriodia=pd.DataFrame()
bacteriodia['date']=[20170412, 20170818, 20171102, 20181030, 20190402, 20190709]
bacteriodia['Bacteroidia_SAGs_/ml']=pd.Series(counts)
bacteriodia['total_cells/ml']=[1545111, 2690141, 1550260, 1452500, 1642105, 2915238]

bacteriodia['%bacteriodia_SAGs']=(bacteriodia['Bacteroidia_SAGs_/ml']/bacteriodia['total_cells/ml'])*100

bacteriodia

,date,Bacteroidia_SAGs_/ml,total_cells/ml,%bacteriodia_SAGs
0,20170412,354615.639344,1545111,22.950820
1,20170818,501891.977612,2690141,18.656716
2,20171102,220657.445255,1550260,14.233577
3,20181030,56258.802817,1452500,3.873239
4,20190402,259597.481618,1642105,15.808824
5,20190709,361891.613793,2915238,12.413793


In [6]:
#metagenome based genus abundance
mg_abundance=pd.read_csv('/mnt/scgc/simon/microg2p/analyses/20210325_GoM_recluster/20210325_GoM_recluster_analysis/GoM_Metagenome_analysis/GoM_metagenome_calculated_Genus_cells_per_ml.csv',
                        index_col=0)


mg_abundance=mg_abundance.merge(classification, left_on='genus', right_on='GTDB_classification', how='left')
mg_Bacteroidiota=mg_abundance[mg_abundance['Phyla']=='Bacteroidota']

mg_Bacteroidiota.drop(mg_Bacteroidiota.columns[[0,1,2,3,9,10,11]], axis=1, inplace=True)

mg_Bacteroidiota

/mnt/scgc/scgc_nfs/opt/common/anaconda3a/envs/jacob_env/lib/python3.6/site-packages/pandas/core/frame.py:4170: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,20171102_genome_size_normalized_metagenome_cells/ml,20181030_genome_size_normalized_metagenome_cells/ml,20190402_genome_size_normalized_metagenome_cells/ml,20190709_genome_size_normalized_metagenome_cells/ml,GTDB_genus,Phyla,Family,GTDB_classification
7,1239.648244,929.992240,7196.501067,5058.403793,Algibacter_B,Bacteroidota,Flavobacteriaceae,Algibacter_B
13,47.855594,21.108130,377.612265,158.251624,Arenitalea,Bacteroidota,Flavobacteriaceae,Arenitalea
15,331.775460,354.795462,1783.197892,630.802091,BACL11,Bacteroidota,BACL11,BACL11
17,6373.493328,2215.349793,382.968226,8171.333221,CAU-1491,Bacteroidota,Flavobacteriaceae,CAU-1491
21,901.054952,777.839163,314.368391,1099.899572,Draconibacterium,Bacteroidota,Prolixibacteraceae,Draconibacterium
23,2911.183411,1514.068364,1514.927357,17140.292617,Flavobacteriaceae,Bacteroidota,Flavobacteriaceae,Flavobacteriaceae
24,1091.358672,655.238549,26545.429480,6124.056645,Flavobacteriales,Bacteroidota,NaN,Flavobacteriales
26,4377.736718,9951.364694,2855.842509,2338.972825,GCA-002715885,Bacteroidota,BACL11,GCA-002715885
29,10404.696606,11904.622256,14774.956393,46848.107223,GCA-002733185,Bacteroidota,Flavobacteriaceae,GCA-002733185
30,11714.118999,11034.720389,1057.341950,2699.395335,GCA-2683775,Bacteroidota,BACL11,GCA-2683775


In [7]:
mg_1711=mg_Bacteroidiota['20171102_genome_size_normalized_metagenome_cells/ml'].sum()
mg_1810=mg_Bacteroidiota['20181030_genome_size_normalized_metagenome_cells/ml'].sum()
mg_1904=mg_Bacteroidiota['20190402_genome_size_normalized_metagenome_cells/ml'].sum()
mg_1907=mg_Bacteroidiota['20190709_genome_size_normalized_metagenome_cells/ml'].sum()
mg_counts=[mg_1711, mg_1810, mg_1904, mg_1907]

mg_bacteriodia=pd.DataFrame()
mg_bacteriodia['date']=[20171102, 20181030, 20190402, 20190709]
mg_bacteriodia['mg_bacteroidia_cells/ml']=pd.Series(mg_counts)
mg_bacteriodia

,date,mg_bacteroidia_cells/ml
0,20171102,527884.042865
1,20181030,266903.604355
2,20190402,531888.275799
3,20190709,902195.790759


In [8]:
bacteriodia=bacteriodia.merge(mg_bacteriodia, on='date', how='left')
bacteriodia['%mg_bacteroidia_cells/ml']=(bacteriodia['mg_bacteroidia_cells/ml']/bacteriodia['total_cells/ml'])*100
bacteriodia

,date,Bacteroidia_SAGs_/ml,total_cells/ml,%bacteriodia_SAGs,mg_bacteroidia_cells/ml,%mg_bacteroidia_cells/ml
0,20170412,354615.639344,1545111,22.950820,NaN,NaN
1,20170818,501891.977612,2690141,18.656716,NaN,NaN
2,20171102,220657.445255,1550260,14.233577,527884.042865,34.051323
3,20181030,56258.802817,1452500,3.873239,266903.604355,18.375463
4,20190402,259597.481618,1642105,15.808824,531888.275799,32.390637
5,20190709,361891.613793,2915238,12.413793,902195.790759,30.947586


In [9]:

chl_concentration['SampleDate']=chl_concentration['SampleDate'].str.replace("-", '')
chl=chl_concentration[['SampleDate', 'ChlWWug/L', 'SampleLocale']].copy()
chl=chl.rename(columns={'SampleDate':'date'})
chl

bacteriodia['date']=bacteriodia['date'].astype(str)
bacteriodia=bacteriodia.merge(chl, on='date', how='outer')
bacteriodia['SampleLocale'].fillna('ebb', inplace=True)
bacteriodia.sort_values(by='date', inplace=True)

bacteriodia=bacteriodia.dropna(axis=0, thresh=3)
bacteriodia=bacteriodia.sort_values(by='date')
bacteriodia

,date,Bacteroidia_SAGs_/ml,total_cells/ml,%bacteriodia_SAGs,mg_bacteroidia_cells/ml,%mg_bacteroidia_cells/ml,ChlWWug/L,SampleLocale
0,20170412,354615.639344,1545111.0,22.950820,NaN,NaN,NaN,ebb
1,20170818,501891.977612,2690141.0,18.656716,NaN,NaN,NaN,ebb
2,20171102,220657.445255,1550260.0,14.233577,527884.042865,34.051323,NaN,ebb
9,20180219,NaN,NaN,NaN,NaN,NaN,0.75,bbh
3,20181030,56258.802817,1452500.0,3.873239,266903.604355,18.375463,1.74,ebb
6,20181205,NaN,NaN,NaN,NaN,NaN,1.84,bbh
7,20181221,NaN,NaN,NaN,NaN,NaN,1.87,bbh
8,20190205,NaN,NaN,NaN,NaN,NaN,0.46,bbh
10,20190319,NaN,NaN,NaN,NaN,NaN,0.72,bbh
4,20190402,259597.481618,1642105.0,15.808824,531888.275799,32.390637,2.91,ebb


In [10]:
df=pd.read_csv('/mnt/scgc/simon/microg2p/analyses/20210325_GoM_recluster/20210325_GoM_recluster_analysis/chl_analysis/DS_chl_measurements.csv')

df[['month', 'day', 'year']] = df.DATE.str.split('/', expand = True)
df['year']=pd.to_numeric(df['year'])+2000
df['year']=df['year'].astype(str)
# add a zero befor the days and months that are a single digit
df['day'] = df['day'].apply(lambda x: x.zfill(2))
df['month'] = df['month'].apply(lambda x: x.zfill(2))

df['date']=df['year']+df['month']+df['day']

df=df.rename(columns={'WW Mean':'ChlWWug/L'})
df=df.drop(columns=['DATE', 'month', 'day', 'year'])
df
#print(df.dtypes)

,ChlWWug/L,date
0,0.99,20081231
1,0.92,20090109
2,1.57,20090114
3,1.61,20090123
4,1.49,20090129
...,...,...
418,6.55,20180723
419,2.70,20180828
420,6.30,20181009
421,1.67,20181205


In [11]:
chl=pd.concat([bacteriodia, df])
chl['year']=chl['date'].str[:4]
chl['month']=chl['date'].str[5:6]
chl['day']=chl['date'].str[6:]
chl['date']=chl['date'].astype(int)
chl=chl[chl['date']>=20170000]
chl=chl[chl['date']<=20200000]
chl['SampleLocale'].fillna('bbh', inplace=True)


chl=chl.sort_values(by='date')
chl.to_csv(outdir+'2017_2019_chla_measurements.csv')
chl=chl.drop(columns=['total_cells/ml',  'year', 'month', 'day'])
chl=chl.rename(columns={'Bacteroidia_SAGs_/ml':'SAG_per_ml', 'mg_bacteroidia_cells/ml': 'mg_per_ml', 'ChlWWug/L':'ChlWW',
                        '%mg_bacteroidia_cells/ml':'perc_mg_bacteroidia', '%bacteriodia_SAGs':'perc_SAGs_bacteroidia'})
chl.to_csv(outdir+'2017_2019_chla_measurements.tsv', sep = '\t', index=False)
chl

,date,SAG_per_ml,perc_SAGs_bacteroidia,mg_per_ml,perc_mg_bacteroidia,ChlWW,SampleLocale
382,20170110,NaN,NaN,NaN,NaN,1.14,bbh
383,20170117,NaN,NaN,NaN,NaN,0.85,bbh
384,20170126,NaN,NaN,NaN,NaN,0.71,bbh
386,20170309,NaN,NaN,NaN,NaN,0.47,bbh
387,20170317,NaN,NaN,NaN,NaN,1.77,bbh
...,...,...,...,...,...,...,...
29,20191015,NaN,NaN,NaN,NaN,6.26,bbh
30,20191107,NaN,NaN,NaN,NaN,9.43,bbh
31,20191113,NaN,NaN,NaN,NaN,4.01,bbh
32,20191125,NaN,NaN,NaN,NaN,1.43,bbh


### I might need to add a column where I calculate the number of RSG bacteriodeta cells to supplement this analysis.

In [12]:
Flavobacteraceae=SAG_abundance[SAG_abundance['Family']=='Flavobacteriaceae']

Flavobacteraceae['1704_cells']=Flavobacteraceae['170412_SAG_abundnance']*cells_1704
Flavobacteraceae['1708_cells']=Flavobacteraceae['170818_SAG_abundnance']*cells_1708
Flavobacteraceae['1711_cells']=Flavobacteraceae['171102_SAG_abundnance']*cells_1711
Flavobacteraceae['1810_cells']=Flavobacteraceae['181030_SAG_abundnance']*cells_1810
Flavobacteraceae['1904_cells']=Flavobacteraceae['190402_SAG_abundnance']*cells_1904
Flavobacteraceae['1907_cells']=Flavobacteraceae['190709_SAG_abundnance']*cells_1907

#sum up all the bacterodiota cells and put into a series
flavo_1704=Flavobacteraceae['1704_cells'].sum()
flavo_1708=Flavobacteraceae['1708_cells'].sum()
flavo_1711=Flavobacteraceae['1711_cells'].sum()
flavo_1810=Flavobacteraceae['1810_cells'].sum()
flavo_1904=Flavobacteraceae['1904_cells'].sum()
flavo_1907=Flavobacteraceae['1907_cells'].sum()
counts=[flavo_1704, flavo_1708, flavo_1711, flavo_1810, flavo_1904, flavo_1907]
counts

/mnt/scgc/scgc_nfs/opt/common/anaconda3a/envs/jacob_env/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/mnt/scgc/scgc_nfs/opt/common/anaconda3a/envs/jacob_env/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/mnt/scgc/scgc_nfs/opt/common/anaconda3a/envs/jacob_env/lib/python3.6/site-packages/

[215302.35245901637,
 271021.6679104478,
 164078.6131386861,
 46029.92957746479,
 193188.82352941178,
 241261.07586206892]

In [13]:
Flavo=pd.DataFrame()
Flavo['date']=[20170412, 20170818, 20171102, 20181030, 20190402, 20190709]
Flavo['SAG_cells/ml']=pd.Series(counts)
Flavo['total_cells/ml']=[1545111, 2690141, 1550260, 1452500, 1642105, 2915238]

Flavo['%Flavo']=(Flavo['SAG_cells/ml']/Flavo['total_cells/ml'])*100

Flavo

,date,SAG_cells/ml,total_cells/ml,%Flavo
0,20170412,215302.352459,1545111,13.934426
1,20170818,271021.667910,2690141,10.074627
2,20171102,164078.613139,1550260,10.583942
3,20181030,46029.929577,1452500,3.169014
4,20190402,193188.823529,1642105,11.764706
5,20190709,241261.075862,2915238,8.275862
